# PIPECAST Weather Forecast Analysis - Complete Demo

## Pipeline Integrated Prediction & Environmental Climate Analysis using Satellite Tracking

This notebook demonstrates the complete PIPECAST workflow:
1. **Forecast Processing** - Fetch weather data and generate Areas of Interest (AOIs)
2. **Enhanced Analysis** - Intersect with census and watershed data
3. **Ensemble Products** - Create probability maps from multiple forecasts
4. **Risk Ranking** - Prioritize AOIs by population exposure

### What You'll Learn
- How to configure forecast parameters
- Process multiple dates and thresholds
- Generate ensemble probability products
- Rank areas by population risk
- Customize for your own use case

In [ ]:
# Install from PyPI
!pip install pipecast-weather

# Verify
import pipecast
print(f"✓ PIPECAST version: {pipecast.__version__}")

## Setup

First, let's import the necessary packages and verify installation.

In [ ]:
# Import PIPECAST
from pipecast import (
    ForecastConfig,
    ForecastProcessor,
    EnsembleProcessor,
    WeatherDataset
)

# Import for analysis
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Verify installation
import pipecast
print(f"✓ PIPECAST version: {pipecast.__version__}")
print("✓ All imports successful!")

## Example 1: Quick Start - Single Day, Single Threshold

Let's start simple: process one day with one threshold to understand the basics.

In [ ]:
# Configure for a quick test
config = ForecastConfig(
    forecast_dates=["2025-02-04"],  # One day
    fxx_list=[12],                  # Just noon forecast
    thresholds=[50],                # 50mm threshold
    forecast_methods=["standard"],  # No enhanced layers (faster)
    weather_dataset="hrrr",         # Continental US
    use_census=False,
    use_watershed=False,
    clip_to_land=False,
    output_dir="./demo_output/example1_quick"
)

print("Configuration:")
print(f"  Dates: {config.forecast_dates}")
print(f"  Forecast hours: {config.fxx_list}")
print(f"  Thresholds: {config.thresholds}")
print(f"  Output: {config.output_dir}")

In [ ]:
# Process the forecast
processor = ForecastProcessor(config)
results = processor.process_all_forecasts()

print("\n✅ Processing complete!")
print(f"Results: {results}")

In [ ]:
# Load and examine the generated AOIs
aoi_files = processor.get_aoi_files()
print(f"Generated {len(aoi_files)} AOI files")

if len(aoi_files) > 0:
    # Load first AOI file
    gdf = gpd.read_file(aoi_files[0])
    print(f"\nFirst AOI file: {aoi_files[0]}")
    print(f"Number of AOIs: {len(gdf)}")
    print(f"\nColumns: {list(gdf.columns)}")
    print(f"\nSample data:")
    display(gdf.head())
    
    # Quick plot
    if len(gdf) > 0:
        fig, ax = plt.subplots(figsize=(12, 8))
        gdf.plot(ax=ax, column='mean_precip_mm', legend=True, cmap='Blues')
        ax.set_title('Areas of Interest - Mean Precipitation > 50mm')
        plt.show()

## Example 2: Multiple Dates and Thresholds

Now let's process multiple days with multiple thresholds - this is more typical for real analysis.

In [ ]:
# Configure for multiple dates and thresholds
config = ForecastConfig(
    forecast_dates=["2025-02-04", "2025-02-05", "2025-02-06"],
    fxx_list=[0, 12, 24],           # Morning, noon, evening
    thresholds=[25, 50, 100],       # Light, moderate, heavy
    forecast_methods=["standard"],
    weather_dataset="hrrr",
    use_census=False,
    use_watershed=False,
    output_dir="./demo_output/example2_multi"
)

processor = ForecastProcessor(config)
results = processor.process_all_forecasts()

print("\n✅ Processed:")
print(f"  {len(config.forecast_dates)} dates")
print(f"  {len(config.fxx_list)} forecast hours")
print(f"  {len(config.thresholds)} thresholds")
print(f"  Total combinations: {len(config.forecast_dates) * len(config.fxx_list) * len(config.thresholds)}")

## Example 3: Enhanced Analysis with Census and Watershed Data

This is where PIPECAST really shines - automatically intersecting forecast AOIs with census population and watershed data.

**Note:** This will download ~470MB of census and watershed data on first run (cached for future use).

In [ ]:
# Configure with enhanced layers
config = ForecastConfig(
    forecast_dates=["2025-02-04", "2025-02-05"],
    fxx_list=[0, 12, 24],
    thresholds=[50, 100],
    forecast_methods=["standard", "enhanced"],  # Both modes
    weather_dataset="hrrr",
    
    # Enable enhanced analysis
    use_census=True,        # Population data
    use_watershed=True,     # Watershed data
    clip_to_land=True,      # Remove ocean forecasts
    
    output_dir="./demo_output/example3_enhanced"
)

print("Enhanced analysis enabled:")
print(f"  ✓ Census population data")
print(f"  ✓ Watershed data")
print(f"  ✓ Land clipping")
print("\nThis will download data on first run (~5 minutes)...")

In [ ]:
# Process with enhanced layers
processor = ForecastProcessor(config)
results = processor.process_all_forecasts()

print("\n✅ Enhanced processing complete!")

In [ ]:
# Compare standard vs enhanced results
date = config.forecast_dates[0]
key = f"F12_T50"

if 'standard' in results and 'enhanced' in results:
    standard = results['standard'][date][key]
    enhanced = results['enhanced'][date][key]
    
    print(f"Results for {date}, F12, 50mm threshold:")
    print("\nStandard:")
    print(f"  AOIs: {standard['num_aois']}")
    print(f"  Mean precip: {standard['mean_precip_over_aois']:.1f}mm")
    
    print("\nEnhanced:")
    print(f"  AOIs: {enhanced['num_aois']}")
    print(f"  Mean precip: {enhanced['mean_precip_over_aois']:.1f}mm")
    print(f"  Population affected: {enhanced.get('census_pop_sum', 0):,}")
    print(f"  Watershed area: {enhanced.get('watershed_area_sum', 0):.2f} deg²")

## Example 4: Ensemble Probability Products

Create ensemble probability maps by aggregating multiple forecasts across dates and forecast hours.

In [ ]:
# First, run a forecast with multiple members
config = ForecastConfig(
    forecast_dates=["2025-02-04", "2025-02-05", "2025-02-06"],
    fxx_list=[0, 6, 12, 18, 24],
    thresholds=[5, 39, 50, 100, 255],
    forecast_methods=["standard"],
    weather_dataset="hrrr",
    output_dir="./demo_output/example4_ensemble"
)

processor = ForecastProcessor(config)
results = processor.process_all_forecasts()

print("\n✅ Forecast processing complete!")
print(f"Total ensemble members: {len(config.forecast_dates) * len(config.fxx_list) * len(config.thresholds)}")

In [ ]:
# Create ensemble probability maps
ensemble = EnsembleProcessor(config.output_dir)
prob_paths = ensemble.create_ensemble_probabilities()

print("\n✅ Ensemble probabilities created!")
print("\nProbability GeoTIFFs:")
for label, path in prob_paths.items():
    print(f"  {label}: {path}")

## Example 5: Risk Ranking by Population

Rank AOIs by population exposure to prioritize disaster response.

In [ ]:
# Rank AOIs by risk
ranked = ensemble.rank_aois_by_probability(
    census_gdf=processor.census_gdf,  # Use loaded census data
    top_n=50
)

print("\n✅ Risk ranking complete!")
print(f"\nTop 10 Highest Risk AOIs:")
display(ranked[['file_name', 'aoi_id', 'bin', 'ensemble_count', 'mean_precip_mm', 'population_affected']].head(10))

In [ ]:
# Visualize risk distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Ensemble count distribution
axes[0].hist(ranked['ensemble_count'], bins=20, edgecolor='black')
axes[0].set_xlabel('Ensemble Count (Probability)')
axes[0].set_ylabel('Number of AOIs')
axes[0].set_title('Distribution of Ensemble Probability')

# Plot 2: Population vs Precipitation
if 'population_affected' in ranked.columns:
    axes[1].scatter(ranked['mean_precip_mm'], ranked['population_affected'], 
                   c=ranked['ensemble_count'], cmap='RdYlBu_r', s=100, alpha=0.6)
    axes[1].set_xlabel('Mean Precipitation (mm)')
    axes[1].set_ylabel('Population Affected')
    axes[1].set_title('Population Exposure vs Precipitation Intensity')
    plt.colorbar(axes[1].collections[0], ax=axes[1], label='Ensemble Count')

plt.tight_layout()
plt.show()

## Example 6: Custom Threshold Bins (Warning Levels)

Define your own warning levels to match your organization's protocols.

In [ ]:
# Configure with custom warning levels
config = ForecastConfig(
    forecast_dates=["2025-02-04"],
    fxx_list=[0, 12, 24],
    
    # Custom thresholds matching your warning system
    thresholds=[10, 25, 50, 75, 100],
    
    # Custom bins for ensemble
    threshold_bins=[
        (0, 10),         # Green - Advisory
        (10, 25),        # Yellow - Watch
        (25, 50),        # Orange - Warning
        (50, 100),       # Red - Severe
        (100, float('inf'))  # Purple - Extreme
    ],
    bin_labels=["Advisory", "Watch", "Warning", "Severe", "Extreme"],
    
    weather_dataset="hrrr",
    output_dir="./demo_output/example6_custom_bins"
)

print("Custom Warning Levels:")
for label, (low, high) in zip(config.bin_labels, config.threshold_bins):
    print(f"  {label:10s}: {low:3.0f} - {high if high != float('inf') else '∞':>3} mm")

In [ ]:
# Process with custom bins
processor = ForecastProcessor(config)
results = processor.process_all_forecasts()

print("\n✅ Processed with custom warning levels!")

## Example 7: Alaska HRRR

Process Alaska forecasts using the HRRR-Alaska dataset.

In [ ]:
# Use preset for Alaska
from pipecast.config import PresetConfigs

config = PresetConfigs.alaska_hrrr(
    forecast_dates=["2025-02-04", "2025-02-05"],
    output_dir="./demo_output/example7_alaska"
)

# Customize
config.fxx_list = [0, 12, 24]
config.thresholds = [50, 100]
config.use_census = True

print(f"Alaska Configuration:")
print(f"  Dataset: {config.weather_dataset.value}")
print(f"  Dates: {config.forecast_dates}")
print(f"  Enhanced: {config.use_census}")

In [ ]:
# Process Alaska
processor = ForecastProcessor(config)
results = processor.process_all_forecasts()

print("\n✅ Alaska processing complete!")

## Example 8: Complete Operational Workflow

This example shows a complete operational workflow from forecast to ranked risk assessment.

In [ ]:
print("="*70)
print("COMPLETE OPERATIONAL WORKFLOW")
print("="*70)

# Step 1: Configure
config = ForecastConfig(
    forecast_dates=["2025-02-04", "2025-02-05", "2025-02-06"],
    fxx_list=[0, 6, 12, 18, 24],
    thresholds=[5, 39, 50, 100, 254, 255],
    forecast_methods=["standard", "enhanced"],
    use_census=True,
    use_watershed=True,
    clip_to_land=True,
    weather_dataset="hrrr",
    output_dir="./demo_output/example8_complete"
)

print("\n>>> Step 1: Configuration")
print(f"  Dates: {len(config.forecast_dates)}")
print(f"  Forecast hours: {len(config.fxx_list)}")
print(f"  Thresholds: {len(config.thresholds)}")
print(f"  Total forecasts: {len(config.forecast_dates) * len(config.fxx_list) * len(config.thresholds)}")

In [ ]:
# Step 2: Process forecasts
print("\n>>> Step 2: Processing forecasts...")
processor = ForecastProcessor(config)
results = processor.process_all_forecasts()
print("✓ Forecast processing complete")

In [ ]:
# Step 3: Create ensemble products
print("\n>>> Step 3: Creating ensemble products...")
ensemble = EnsembleProcessor(config.output_dir)
prob_paths = ensemble.create_ensemble_probabilities()
print(f"✓ Created {len(prob_paths)} probability maps")

In [ ]:
# Step 4: Rank by risk
print("\n>>> Step 4: Ranking AOIs by population risk...")
ranked = ensemble.rank_aois_by_probability(
    census_gdf=processor.census_gdf,
    top_n=100
)
print(f"✓ Ranked {len(ranked)} AOIs")

In [ ]:
# Step 5: Generate report
print("\n" + "="*70)
print("OPERATIONAL SUMMARY")
print("="*70)

print(f"\nForecast Period: {config.forecast_dates[0]} to {config.forecast_dates[-1]}")
print(f"Total AOI files generated: {len(processor.get_aoi_files())}")
print(f"Ensemble probability maps: {len(prob_paths)}")

if 'population_affected' in ranked.columns:
    total_pop = ranked['population_affected'].sum()
    print(f"\nTotal population in high-risk areas: {total_pop:,}")
    print(f"\nTop 5 Highest Risk Areas:")
    display(ranked[['file_name', 'bin', 'ensemble_count', 'mean_precip_mm', 'population_affected']].head(5))
else:
    print(f"\nTop 5 Areas by Ensemble Probability:")
    display(ranked[['file_name', 'bin', 'ensemble_count', 'mean_precip_mm']].head(5))

print(f"\nOutputs saved to: {config.output_dir}")
print("\n✅ WORKFLOW COMPLETE")

## Summary

You've now learned how to:

✅ **Configure forecasts** with dates, thresholds, and datasets  
✅ **Process weather data** to generate AOIs  
✅ **Enable enhanced analysis** with census and watershed data  
✅ **Create ensemble products** from multiple forecasts  
✅ **Rank by risk** to prioritize response  
✅ **Customize** for your specific use case  

## Next Steps

1. **Try with your own dates** - Use recent dates for current conditions
2. **Adjust thresholds** - Match your organization's warning levels
3. **Add custom layers** - Include pipelines, infrastructure, etc.
4. **Automate** - Run regularly for operational forecasting
5. **Integrate** - Use outputs in your GIS or analysis workflows

## Documentation

- **README.md** - Complete package documentation
- **QUICKSTART_COLAB.md** - Google Colab guide
- **examples/complete_example.py** - All examples in one script

## Support

- GitHub Issues: [Report problems](https://github.com/NASA-EarthRISE/PIPECAST/issues)
- Documentation: Check README.md for detailed information